In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
import nltk
from model import TrieNode

In [2]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
    def disp_plus(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            if child.count > 10:
                child.disp_plus(ind+1)

In [3]:
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
def updateFPtree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 判断items的第一个结点是否已作为子结点
        inTree.children[items[0]].inc(count)
    else:
        # 创建新的分支
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新相应频繁项集的链表，往后添加
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # 递归
    if len(items) > 1:
        updateFPtree(items[1::], inTree.children[items[0]], headerTable, count)

def createFPtree(word_dict, cutted, minSup=1):
    headerTable = {}
    for trans in word_dict:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    freqItemSet = set(headerTable.keys()) # 满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] # element: [count, node]

    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        # dataSet：[element, count]
        localD = {}
        for item in tranSet:
            if item in freqItemSet: # 过滤，只取该样本中满足最小支持度的频繁项
                localD[item] = headerTable[item][0] # element : count
        if len(localD) > 0:
            # 根据全局频数从大到小对单样本排序
            orderedItem = [v[0] for v in localD.items()]
            # 用过滤且排序后的样本更新树
            updateFPtree(orderedItem, retTree, headerTable, count)
    return retTree, headerTable

In [5]:
def shorten(s):
    ss= ''.join(reversed(s))
    k = -1
    j = 0
    array = [-1]
    plen = len(ss)
    zeros = 0
    nonze = 0
    while j < plen:
        if k == -1 or ss[j] == ss[k]:
            k += 1
            j += 1
            if k == 0:
                if j != zeros + 1:
                    loop = (int)(nonze/zeros) -1
                    return cut_string(s, plen) if loop<1 else cut_string(s, plen - zeros*loop)
                zeros += 1
            else:
                nonze += 1
            array.append(k)
        else:
            k = array[k]
    return cut_string(s, plen) if zeros==plen else (cut_string(s, zeros) if zeros>1 else cut_string(s, zeros+1)) 
def cut_string(s, lens):
    return s[0: lens]

In [6]:
def loadSimpDat():
    word_dict={}
    with open('danmu_9.txt','r', newline='\n',encoding = 'gbk') as f:
        lines = f.readlines()
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if word_dict.get(line):
            word_dict[line] += 1
        else:
            word_dict[line] = 1
    return word_dict
# 构造成 element : count 的形式
def createInitSet(dataSet):
    retDict={}
    for trans in dataSet:
        if key in retDict:
            retDict[trans] += 1
        else:
            retDict[trans] = 1
    return retDict

word_dict = loadSimpDat()
#initSet = createInitSet(simDat)
#print(initSet)
#myFPtree, myHeaderTab = createFPtree(initSet, 3) # 最小支持度3
#myFPtree.disp()

In [7]:
sorted(word_dict.items(), key = lambda item:item[1], reverse = True)

[('卖炭翁', 40),
 ('哈哈', 37),
 ('百鬼丸', 36),
 ('满面尘灰烟火色', 35),
 ('泪目', 32),
 ('两鬓苍苍十指黑', 26),
 ('前方高能', 26),
 ('awsl', 25),
 ('flag', 22),
 ('犹豫就会败北', 19),
 ('工具人', 17),
 ('好看吗', 17),
 ('果断就会白给', 17),
 ('追了', 16),
 ('护食', 16),
 ('恶鬼灭杀', 16),
 ('好帅', 15),
 ('话疗', 15),
 ('有故事', 14),
 ('啦啦', 14),
 ('好看', 14),
 ('伐薪烧炭南山中', 13),
 ('俺没偷胖次', 13),
 ('只狼', 12),
 ('碳只狼', 12),
 ('考哥', 12),
 ('爱了', 11),
 ('摸头杀', 10),
 ('贝吉塔', 10),
 ('心疼声优', 10),
 ('emm', 10),
 ('哦豁', 10),
 ('哭了', 10),
 ('全剧终', 10),
 ('喜之郎', 9),
 ('完结撒花', 9),
 ('菜是原罪', 9),
 ('可怜身上衣正单', 9),
 ('我家还蛮大的', 8),
 ('完蛋', 8),
 ('这番我追定了', 8),
 ('强行解说', 8),
 ('吹爆', 8),
 ('flag立的飞起', 8),
 ('真能生', 8),
 ('好可怕', 8),
 ('啊啊', 8),
 ('长兄如父', 8),
 ('牛困人饥日已高', 7),
 ('心疼', 7),
 ('经费在燃烧', 7),
 ('雪中送炭', 7),
 ('夜来城外一尺雪', 7),
 ('全员兄控', 7),
 ('画质感人', 7),
 ('逢鬼必斩', 7),
 ('完了', 7),
 ('猎魔人', 7),
 ('炭只狼', 7),
 ('尸鬼', 7),
 ('身上衣裳口中食', 7),
 ('我哭了', 6),
 ('好帅啊', 6),
 ('晓驾炭车碾冰辙', 6),
 ('水柱', 6),
 ('被偷家了', 6),
 ('阿姨压一压', 6),
 ('败者食尘', 6),
 ('内心戏真多', 6),
 ('遗照', 6),
 ('宇智

In [8]:
for word in word_dict:
    if word_dict[word] >5:
        print(word, word_dict[word])

我哭了 6
好帅啊 6
晓驾炭车碾冰辙 6
百鬼丸 36
满面尘灰烟火色 35
好帅 15
flag 22
有故事 14
工具人 17
喜之郎 9
泪目 32
awsl 25
只狼 12
水柱 6
追了 16
被偷家了 6
阿姨压一压 6
护食 16
啦啦 14
爱了 11
好看吗 17
牛困人饥日已高 7
话疗 15
恶鬼灭杀 16
哈哈 37
我家还蛮大的 8
摸头杀 10
碳只狼 12
心疼 7
犹豫就会败北 19
经费在燃烧 7
败者食尘 6
内心戏真多 6
两鬓苍苍十指黑 26
完结撒花 9
雪中送炭 7
贝吉塔 10
夜来城外一尺雪 7
心疼声优 10
完蛋 8
这番我追定了 8
全员兄控 7
emm 10
哦豁 10
画质感人 7
逢鬼必斩 7
强行解说 8
前方高能 26
完了 7
遗照 6
吹爆 8
宇智波斑 6
果断就会白给 17
flag立的飞起 8
菜是原罪 9
伐薪烧炭南山中 13
哭了 10
好看 14
猎魔人 7
什么鬼 6
炭只狼 7
多罗罗 6
俺没偷胖次 13
考哥 12
真能生 8
好可怕 8
啊啊 8
尸鬼 7
长兄如父 8
卖炭翁 40
可怜身上衣正单 9
漫漫长夜 6
樱井孝宏 6
Amazon 6
你家没了 6
全剧终 10
身上衣裳口中食 7
嘴遁 6
阿帕茶 6
高能 6
妖魔鬼怪快离开 6


In [9]:
headerTable = {}
for trans in word_dict:
    for item in trans:
        headerTable[item] = headerTable.get(item, 0) + word_dict[trans]
freqItemSet = set(headerTable.keys())
if len(freqItemSet) == 0:
    print('666')
for k in headerTable:
    headerTable[k] = [headerTable[k], None] # element: [count, node]
retTree = treeNode('Null Set', 1, None)
for tranSet, count in word_dict.items():
        # dataSet：[element, count]
    localD = {}
    for item in tranSet:
        if item in freqItemSet:
            localD[item] = headerTable[item][0] # element : count
    if len(localD) > 0:
        orderedItem = [v[0] for v in localD.items()]
        updateFPtree(orderedItem, retTree, headerTable, count)

In [10]:
next_tree = headerTable['点'][1]
print(next_tree.count)
print(len(next_tree.children))
print(next_tree.nodeLink.count)

1
0
1


In [11]:
def addNode(retTree, child):
    if child.name in retTree.children:
        retTree.children[child.name].inc(child.count)
    else:
        retTree.children[child.name] = treeNode(child.name, child.count, retTree)
    for child2 in child.children.values():
        if child2 is not None:
            addNode(retTree.children[child.name], child2)

In [12]:
def new_word(retTree, child, word, new_words):
    if child is not None and child.count> 6 and (child.count / retTree.count)>0.5:
        for child2 in child.children.values():
            new_word_l(child, child2, word + child.name, new_words)
        else:
            print(child is not None, child.count> 6)
            new_words.add(word + child.name)
            return

In [13]:
def value(list1, list2):
    list3 = [i for i in list1 if i in list2]
    return len(list3)*1.0/(len(list1)+len(list2)-1)

In [14]:
def ngram(word, n=2):
    grams = []
    for i in range(len(word)-n+1):
        grams.append(word[i:i+n])
    return grams

In [16]:
import math
def new_word_l(retTree, child, word, headerTable):
    target = word
    #print(retTree.name,retTree.count, child.name,child.count,(child.count / retTree.count)>0.3)
    if child is not None and child.count > 10 and ((child.count / retTree.count) > 1.6 - 0.4 * math.log(child.count) or headerTable[child.name][0] < child.count+2):
        for child2 in child.children.values():
            word1 = new_word_l(child, child2, word + child.name, headerTable)
            if len(word1) > len(target):
                target = word1
    return target

In [15]:
def simplify(words):
    new_words = [words[0]]
    for word in words[1:]:
        word = word.lower()
        flag = True
        w = 0
        g1 = ngram(word)
        while w < len(new_words) and flag:
            g2 = ngram(new_words[w])
            try: 
                if word in new_words[w]:
                    flag = False
                elif value(g1, g2) > 0.1:
                    print(word, new_words[w], value(g1, g2))
                    flag = False
            except ZeroDivisionError:
                flag = False
            w += 1
        if flag:
            new_words.append(word)
    word = []
    for w in new_words:
        if len(w) < 5:
            word.append(w)
    return word

In [29]:
travel = set()
new_words = set()
for selected_word in headerTable:
    retTree = treeNode(selected_word, headerTable[selected_word][0], None)
    next_tree = headerTable[selected_word][1]
    while next_tree is not None:
        for child in next_tree.children.values():
            addNode(retTree, child)
        next_tree = next_tree.nodeLink
    new_word = selected_word
    if retTree.parent is not None and retTree.count/headerTable[selected_word][0] > 0.7 and retTree.count > 10:
        new_word = retTree.parent.name + new_word
    for child in retTree.children.values():
        result = new_word_l(retTree, child, new_word, headerTable)
        travel.add(child.name)
        if len(result) > 1:
            new_words.add(result)
new_words = sorted(list(new_words),key = lambda i:len(i),reverse=True)

In [30]:
simplify(new_words)

犹豫就会败 果断就会白 0.14285714285714285
豫就会败 果断就会白 0.16666666666666666
就会败 果断就会白 0.2


['恶鬼灭杀',
 '只是个卖',
 'flag',
 '宇智波斑',
 '前方高能',
 '俺没偷胖',
 '一家人',
 'wsl',
 'bgm',
 '工具人',
 '强行解',
 '摸头杀',
 '卖炭翁',
 '偷家了',
 '出来挨',
 '洗海带',
 '为什么',
 '这么多',
 '祢豆子',
 '内心戏',
 '太多了',
 '应该是',
 '的声优',
 '电线杆',
 '百鬼丸',
 '剧情',
 '解说',
 '泪目',
 '熟悉',
 '衣服',
 '这是',
 '开头',
 '厉害',
 '辛苦',
 '延迟',
 '鸣人',
 '疯狂',
 '男主',
 '够了',
 '治愈',
 '多罗',
 '尸鬼',
 '紫伞',
 '卖碳',
 '既视',
 '一样',
 '有点',
 '的好',
 '画质',
 '哦豁',
 '就是',
 '经费',
 '好看',
 '感觉',
 '大叔',
 '世界',
 '居然',
 '水晶',
 '东西',
 '莫名',
 '想到',
 '飞碟',
 '苇名',
 '已经',
 '母亲',
 '现在',
 '喜欢',
 '温柔',
 '只狼',
 '色的',
 '僵尸',
 '耳朵',
 '视感',
 '刺猬',
 '细节',
 '拔刀',
 '知道',
 '一个',
 '23',
 '佐助',
 '响鬼',
 '所以',
 '了吧',
 '因为',
 '话疗',
 '花子',
 '可以',
 '节奏',
 '时候',
 '丧尸',
 '胖次',
 '营养',
 '斧头',
 '需要',
 '以为',
 '哭了',
 '吹爆',
 '耳环',
 '任务',
 '山中',
 '到了',
 '导师',
 '追了',
 '这个',
 '不是',
 '可爱',
 '开始',
 '死了',
 '也太',
 '声音',
 '其实',
 '又是',
 '几个',
 '格子',
 '姿势',
 '没有',
 '可能',
 '漂亮',
 '犬病',
 '蛮大',
 '洛特',
 '自己',
 '晚上',
 '肯定',
 '眼睛',
 '计划',
 '是你',
 '心疼',
 '考哥',
 '闪现',
 '怎么',
 '都是',
 '梅林',
 '夏树',
 '乌鸦',
 '吓人',
 '突然',
 '恐怖',
 'l

In [209]:
'lag立得飞起' in 'flag立得飞起'

True

In [37]:
selected_word = '两'
retTree = treeNode(selected_word, headerTable[selected_word][0], None)
next_tree = headerTable[selected_word][1]
while next_tree is not None:
    for child in next_tree.children.values():
        addNode(retTree, child)
    next_tree = next_tree.nodeLink
#retTree.disp_plus()
new_words = set()

In [38]:
retTree.disp_plus()

   两   61
     鬓   32
       苍   28
         十   28
           指   28
             黑   28


In [32]:
new_word = ''
for child in retTree.children.values():
    new_word = new_word_l(retTree, child, retTree.name, new_word)

In [33]:
new_word

'几'

In [121]:
retTree.disp_plus()

   取   17
     消   15
       了   12


In [96]:
next_tree = headerTable['大'][1]
next_tree.children.values()

dict_values([<__main__.treeNode object at 0x0000026FFDDCF488>, <__main__.treeNode object at 0x0000026FFDE30F48>, <__main__.treeNode object at 0x0000026FFDEE4108>, <__main__.treeNode object at 0x0000026FFDF60408>, <__main__.treeNode object at 0x0000026FFDF80148>])

In [66]:
headerTable['金'][0]

31

In [80]:
next_tree = headerTable['荣'][1]
while next_tree is not None:
    next_tree.disp()
    next_tree = next_tree.nodeLink

   荣   17
     誉   3
       永   3
         远   3
           属   3
             于   3
               星   3
                 辰   3
                   十   3
                     字   3
                       军   3
     耀   14
       永   9
         远   9
           属   9
             于   9
               星   9
                 尘   5
                   十   4
                     字   4
                       军   4
                   军   1
                 辰   4
                   斗   1
                     士   1
                   十   3
                     字   3
                       军   3
       属   5
         于   5
           星   5
             尘   3
               十   3
                 字   3
                   军   3
             辰   2
               十   2
                 字   2
                   军   2


In [44]:
retTree.disp()

   Null Set   1
     完   4
       结   4
         撒   3
           花   3
         啦   1
     黄   3
       金   3
         精   3
           神   3
     大   8
       乔   8
         停   1
           止   1
             了   1
               思   1
                 考   1
         又   2
           停   1
             止   1
               了   1
                 思   1
                   考   1
           点   1
             了   1
               赞   1
         不   1
           知   1
             道   1
               应   1
                 该   1
                   点   1
                     赞   1
                       还   1
                         是   1
                           沉   1
                             默   1
         为   1
           你   1
             点   1
               赞   1
         点   1
           了   1
             个   1
               踩   1
         有   1
           给   1
             你   1
               点   1
                 了   1
                   一   1
                     个

In [97]:
orderedItem

['意', '大', '利', '的', '奇', '妙', '服', '装']

In [40]:
print ("/".join(jieba.cut("大乔为你点赞")))

大乔为/你/点赞


In [41]:
def loadSimpDat():
    word_dict={}
    with open('danmu_9.txt','r', newline='\n',encoding = 'gbk') as f:
        lines = f.readlines()
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if word_dict.get(line):
            word_dict[line] += 1
        else:
            word_dict[line] = 1
    return word_dict
word_dict = loadSimpDat()

In [42]:
for line in word_dict:
    print ("/".join(jieba.cut(line)))

心疼/妹妹/声优
你/比较/好吃
卖/碳/的/小/蓝孩
芳华/人设
目测/是/好/番
我莫得/感情
画质/太棒了
我/哭/了
猎妈者
已/过半
考哥/呀
好帅/啊
开局/flag/绝/了
莽夫
体液/传播
北杨克顿/的/慢/一点
花里胡哨/的
我/慌/了
教科书/一样/的/flage
可怕
no
bu
我/开始/慌/了
正义/的/伙伴
flag/是/个/好/东西
都/是/薇/兄控/的
闪现
晓驾/炭车/碾冰辙
妹妹/吃/啥/啊
承太郎
阿伟/死/了
居合
啥/是/flag
主角/要/变成/林正英/了/吗
好/漂亮
秀飞
大概/是/咋/骂/自己/了
感觉/好好看/的/样子
一拳/超人/乱入
响转
nb
真/TM/话/多
哪有/这么/多/心理/对白
来/点/阿莫西林
白眼/觉醒
砸瓦路/多
BGM/什么/鬼
真灵
这/怎么/生/的/啊
好/卑微
诈尸
出事
人鬼/殊途
妹夫
一只/小/团团
名/侦探/碳治郎
长夜漫漫/式/开头
百鬼丸
山风/可太/真实/了
心动/的/感觉
哦/豁百鬼/玩
强人/锁/男
头发/好好看
满面/尘灰/烟火/色
风紧/扯/呼
妈妈/好/矮/哎
神秘/湖
卖/碳/翁
路鸣泽
好帅
flag
话/有点/多
好/强势
大陆/版/这里/加/了/黑/滤镜
剧情简介/怎么/有点像/薄樱鬼
看/斧头
帮/装备
大家/变色/吧
这腿/好/怪/啊
我/还/以为/觉醒/了/呢
话/太/多
这么/能生
金刚/芭比
卖炭/娃
接下来/丧尸/缓解
有/故事
工具/人
回去/就/玩完
正经/点别/恶搞
好运
six
插满/了/flag
我/的/天/这/一家子/人/还/挺/多
七口/之/家
德国/骨科
我莫得/感情/的/杀手
请/务必/带/她/去
乘热
山歌
是/说谎/的/味道
罗罗多
出货/了
喜之郎
连续/按/X/挣脱
有/脚步
战术/分析/哈哈
标题/正入
应该/不是/全尸/了/已经
想到/无名
突然/觉得/这/两人/好配
指甲/真的/好看
听到/了/9s
下/一个/是不是/侯府/摘/女
岸边/露伴
这/鸽子/演/的/太好了/吧
耳坠/好帅
耳坠/不成
什么/类型
HASAKI
上古/卷轴/视感
妹妹/被/人/抢/了
牛皮
我/是/个/没有/没/得/感情/的/杀手
男主/的/家/可能/没有/了
帅气/啊
救/了/个/对象
恐怖
吃/的/还/挺/丰盛
系向/碳头/充牛值